In [2]:
#Required packages
import numpy as np
import pandas as pd
from riotwatcher import LolWatcher, ApiError
import requests
from bs4 import BeautifulSoup

In [3]:
key = 'RGAPI-8466775c-bb66-482b-a766-cf1361ecab03'
watcher = LolWatcher(key)
my_region = 'na1'

In [3]:
#me = watcher.summoner.by_name(my_region, 'Doublelift')
#my_ranked_stats = watcher.league.by_summoner(my_region, me['id'])

In [10]:
# Scraping individual champion score
r1 = requests.get("https://www.metasrc.com/5v5/na/tierlist")
soup1 = BeautifulSoup(r1.text, "html.parser")
power_level = soup1.findAll("div", {"class": " _9581uw"})
names = soup1.findAll("div", {"class": " _q8ue62"})
champ = []
for name in names:
    champ.append(name.getText())
rates = []
for rate in power_level:
    rates.append(rate.getText())
tierlist = pd.DataFrame().assign(
    champ = champ,
    score = rates
)
tierlist.sort_values(by = ["champ"])

# Getting individual Champion IDs
IDs = pd.read_csv("new.txt", sep = ":", header = None)
IDs.columns = ["ID", "champ"]

champion_list = tierlist.sort_values(by = ["champ"])["champ"].unique()

# Merging Ids and individual champion score
new_tierlist = []

# Averaging similar champions scores into one
for champ in champion_list:
    row = {}
    champ_list = tierlist[tierlist["champ"] == champ]
    champ_list_avg = champ_list["score"].astype(float).mean()
    row["champ"] = champ
    row["score"] = champ_list_avg
    new_tierlist.append(row)

tierlist_pd = pd.DataFrame(new_tierlist)
tierlist_pd = tierlist_pd.sort_values(by = ["champ"])
champion_score = IDs.merge(tierlist_pd, on = champion_list)
champion_score = champion_score.drop(columns = ["champ_x", "champ_y"])
champion_score.columns = ["champion", "champion_id", "score"]
champion_score = champion_score.set_index(["champion_id"])
champion_score

,champion,score
champion_id,,
266,Aatrox,47.030
103,Ahri,60.900
84,Akali,37.040
12,Alistar,51.230
32,Amumu,52.380
...,...,...
238,Zed,68.430
115,Ziggs,40.385
26,Zilean,47.815


In [70]:
gameIDs = [
 3450384967,
 3449941896,
 3449917039,
 3449911789,
 3449857306,
 3449804628,
 3449616597,
 3449619982,
 3449403502,]

input_matches = []

for ID in gameIDs:
    match_info = watcher.match.by_id(my_region, ID)
    input_matches.append(match_info)

In [71]:
# from matchID to matches

# Inputting data frame
output_detail = []
y_result = []

difference_list = ["totalMinionsKilled", "longestTimeSpentLiving", "totalHeal", "totalDamageDealt", "magicDamageDealt", "physicalDamageDealt", "totalDamageDealtToChampions", "magicDamageDealtToChampions", "trueDamageDealtToChampions", "goldEarned"]
tower_list = ["towerKills", "inhibitorKills", "riftHeraldKills"]

#Creating individual player scores
for match_detail in input_matches:
    participants = []
    team = []
    scores = []
    
    print(match_detail["gameMode"])
    # Calculate Individual statistics
    for row in match_detail['participants']:
        participants_row = {}
        participants_row['champion_Id'] = row['championId']
        participants_row['team'] = row['teamId']
        participants_row['kills'] = row['stats']['kills']
        participants_row['deaths'] = row['stats']['deaths']
        participants_row['assists'] = row['stats']['assists']
        participants_row['totalDamageDealt'] = row['stats']['totalDamageDealt']
        participants_row['magicDamageDealt'] = row['stats']['magicDamageDealt']
        participants_row['physicalDamageDealt'] = row['stats']['physicalDamageDealt']
        participants_row['physicalDamageDealt'] = row['stats']['physicalDamageDealt']
        participants_row['totalDamageDealtToChampions'] = row['stats']['totalDamageDealtToChampions']
        participants_row['magicDamageDealtToChampions'] = row['stats']['magicDamageDealtToChampions']
        participants_row['physicalDamageDealtToChampions'] = row['stats']['physicalDamageDealtToChampions']
        participants_row['trueDamageDealtToChampions'] = row['stats']['trueDamageDealtToChampions']    
        participants_row['goldEarned'] = row['stats']['goldEarned']
        participants_row['champLevel'] = row['stats']['champLevel'] / 18
        participants_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
        participants_row['largestMultiKill'] = row['stats']['largestMultiKill']
        participants_row['killingSprees'] = row['stats']['killingSprees']
        participants_row['doubleKills'] = row['stats']['doubleKills']
        participants_row['tripleKills'] = row['stats']['tripleKills']
        participants_row['quadraKills'] = row['stats']['quadraKills']
        participants_row['pentaKills'] = row['stats']['pentaKills']
        participants_row['longestTimeSpentLiving'] = row['stats']['longestTimeSpentLiving']
        participants_row['totalHeal'] = row['stats']['totalHeal']
        participants_row['damageDealtToObjectives'] = row['stats']['damageDealtToObjectives']
        participants_row['damageDealtToObjectives'] = row['stats']['damageDealtToObjectives']
        participants_row['damageDealtToTurrets'] = row['stats']['damageDealtToTurrets']
        participants_row['visionScore'] = row['stats']['visionScore']
        participants_row['timeCCingOthers'] = row['stats']['timeCCingOthers']
        participants_row['totalDamageTaken'] = row['stats']['totalDamageTaken']
        participants_row['magicalDamageTaken'] = row['stats']['magicalDamageTaken']
        participants_row['physicalDamageTaken'] = row['stats']['physicalDamageTaken']   
        participants_row['trueDamageTaken'] = row['stats']['trueDamageTaken']   
        participants.append(participants_row)

    # Champion scores conversion
    match = pd.DataFrame(participants)
    champions = match["champion_Id"]
    for champ in champions:
        scores.append(champion_score.loc[champ].get("score"))
    scores_pd = pd.DataFrame(scores)
    scores_pd.columns = ["score"]
    match = match.join(scores_pd)
    grouped = match.groupby(["team"]).sum()
    grouped = grouped.drop(columns = ['champion_Id'])
        
    # Calculate Team statistsics
    for row in match_detail['teams']:
        team_row = {}
        team_row['team'] = row['teamId']
        team_row['firstBlood'] = int(row['firstBlood'] == True)
        team_row['firstTower'] = int(row['firstTower'] == True)
        team_row['firstBaron'] = int(row['firstBaron'] == True)
        team_row['firstDragon'] = int(row['firstDragon'] == True)
        team_row['firstBlood'] = int(row['firstBlood'] == True) 
        team_row['firstRiftHerald'] = int(row['firstRiftHerald'] == True)
        team_row['towerKills'] = row['towerKills'] / 11
        team_row['inhibitorKills'] = row['inhibitorKills'] / 3
        team_row['baronKills'] = row['baronKills'] 
        team_row['dragonKills'] = row['dragonKills']    
        team_row['riftHeraldKills'] = row['riftHeraldKills'] / 2
        result = row["win"]
        if (result == "Win"):
            y_result.append("1")
        elif (result == "Fail"):
            y_result.append("0")
        team.append(team_row)

    team_df = pd.DataFrame(team)
    final_df = team_df.merge(grouped, on = "team")
    final_df = final_df.drop(columns = ["team"])
    #print(final_df)

    #for column in difference_list:
        
    #break;
    for index, row in final_df.iterrows():
        output_detail.append(row)
        

ARAM
CLASSIC
CLASSIC
CLASSIC
CLASSIC
CLASSIC
CLASSIC
CLASSIC
CLASSIC
CLASSIC


In [61]:
output_detail

[firstBlood                             1.000000
 firstTower                             0.000000
 firstBaron                             0.000000
 firstDragon                            0.000000
 firstRiftHerald                        0.000000
 towerKills                             0.363636
 inhibitorKills                         0.333333
 baronKills                             0.000000
 dragonKills                            0.000000
 riftHeraldKills                        0.000000
 kills                                 49.000000
 deaths                                42.000000
 assists                              122.000000
 totalDamageDealt                  369953.000000
 magicDamageDealt                  234939.000000
 physicalDamageDealt               112867.000000
 totalDamageDealtToChampions       142775.000000
 magicDamageDealtToChampions        73760.000000
 physicalDamageDealtToChampions     57194.000000
 trueDamageDealtToChampions         11818.000000
 goldEarned         